In [19]:
#load all libraries
import nltk
import numpy as np
from nltk.corpus import wordnet as wn

In [21]:
#bypasses certification issue
#comment this out if you don't have any problem
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

#download WordNet
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /Users/deeksha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [48]:
#check for an example
#from nltk.corpus import wordnet
synsets = wn.synsets('dog')

#check for definition of "dog"
for synset in synsets:
    print(synset.definition())

a member of the genus Canis (probably descended from the common wolf) that has been domesticated by man since prehistoric times; occurs in many breeds
a dull unattractive unpleasant girl or woman
informal term for a man
someone who is morally reprehensible
a smooth-textured sausage of minced beef or pork usually smoked; often served on a bread roll
a hinged catch that fits into a notch of a ratchet to move a wheel forward or prevent it from moving backward
metal supports for logs in a fireplace
go after with the intent to catch


In [49]:
#function for creating a semantic lexicon dictionary 
def semantic_lexicon_dict(target_words):
    #dictionary to store the related words for each target word
    #keys = target words
    #values = related words
    lexicon = {}

    #Use WordNet to find related words through various semantic relations for each target word
    #loop over the target words
    for word in target_words:
        #initialise empty related_words list which will hold 
        #all the semantically related words extracted from WordNet
        related_words = []
        #iterate through each synset (set of synonyms)
        for syn in wn.synsets(word):
            #iterate thorugh each lemma for the synset and append to related_words
            for lemma in syn.lemmas():
                if lemma.name() != word:
                    related_words.append(lemma.name())
            #check antonym for first lemma of the synset and append
            if syn.lemmas()[0].antonyms():
                related_words.append(syn.lemmas()[0].antonyms()[0].name())
            #check for hyponym and append
            for hypo in syn.hyponyms():
                for lemma in hypo.lemmas():
                    related_words.append(lemma.name())
            #check for hypernym and append
            for hyper in syn.hypernyms():
                for lemma in hyper.lemmas():
                    related_words.append(lemma.name())
            #check for meronym and append
            for part in syn.part_meronyms():
                for lemma in part.lemmas():
                    related_words.append(lemma.name())
            #check for holonym and append
            for whole in syn.part_holonyms():
                for lemma in whole.lemmas():
                    related_words.append(lemma.name())
            #iterate through each lemma for the current synset
            #for each lemma not the same as target word
            #find all lemmas that have same spelling and append
            for lemma in syn.lemmas():
                if lemma.name() != word:
                    homonyms = wn.lemmas(lemma.name())
                    for homonym in homonyms:
                        related_words.append(homonym.name())
        lexicon[word] = related_words
    return lexicon

In [62]:
#function for creating a semantic lexicon matrix
def semantic_lexicon_mat(semantic_lexicon_dictionary, target_words_list, NEW_ROWS, NEW_COLUMNS):
    #a set (set because duplicates are removed) of unique semantic relations
    #these relations will serve as columns in the matrix
    semantic_relations = set()
    for rel_word in semantic_lexicon_dictionary.values():
        semantic_relations.update(rel_word)
    
    #Create a matrix filled with zeroes
    #Each row represents a target word
    #Each column represents a semantic relation
    ROWS = target_words_list
    COLUMNS = semantic_relations
    semantic_lexicon_matrix = np.zeros((len(ROWS), len(COLUMNS)), dtype=int)
    
    #loop over the target_words_list and gets the word and its index
    for i, target_word in enumerate(target_words_list):
        #retrieve semantic related words for current target word from the semantic_lexicon_dict
        related_words = semantic_lexicon_dictionary.get(target_word, [])
        #loop over each related word for the current target word
        for related_word in related_words:
            #get the index of the related word from the semantic_relations
            #converted to list so that index() can be used
            j = list(semantic_relations).index(related_word)
            #set the element (i, j) to 1 to indicate the relationship between
            #the target word and the semantically related word
            semantic_lexicon_matrix[i, j] = 1
    
    return semantic_lexicon_matrix


In [63]:
#Define the target words (sentiment analysis task)

target_words = ['good', 'bad', 'sad', 'happy', 'awesome', 'scary']

#Create the semantic lexicon dictionary by passing the target words as argument
semantic_lexicon_dictionary = semantic_lexicon_dict(target_words)

#prints the semantic lexicon dictionary
print(semantic_lexicon_dictionary)

#check the semantically related words for "bad"
print(semantic_lexicon_dictionary.get("bad"))

{'good': ['common_good', 'commonweal', 'advantage', 'vantage', 'goodness', 'evil', 'beneficence', 'benignity', 'benignancy', 'graciousness', 'kindness', 'saintliness', 'summum_bonum', 'virtue', 'virtuousness', 'moral_excellence', 'virtue', 'morality', 'goodness', 'goodness', 'goodness', 'bad', 'benefit', 'welfare', 'better', 'better', 'desirability', 'desirableness', 'optimum', 'wisdom', 'wiseness', 'soundness', 'worthiness', 'quality', 'goodness', 'goodness', 'commodity', 'trade_good', 'basic', 'staple', 'consumer_goods', 'drygoods', 'soft_goods', 'entrant', 'export', 'exportation', 'fancy_goods', 'fungible', 'future', 'import', 'importation', 'merchandise', 'ware', 'product', 'middling', 'salvage', 'shopping', 'sporting_goods', 'worldly_possession', 'worldly_good', 'artifact', 'artefact', 'commodity', 'trade_good', 'bad', 'full', 'full', 'full', 'full', 'full', 'full', 'full', 'full', 'full', 'full', 'full', 'full', 'full', 'full', 'evil', 'estimable', 'honorable', 'respectable', 'es

In [67]:
#uses the same semantic lexicon dictionary created earlier
#assigned to semantic_lexicon just for readability
semantic_lexicon = semantic_lexicon_dictionary

#adjust dimensionality of matrix
'''dimensionality to be manipulated here to match word embeddings'''
NEW_ROWS = 100
NEW_COLUMNS = 150

#create the semantic lexicon matrix by passing the semantic lexicon dictionary and target words as arguments
semantic_lexicon_matrix = semantic_lexicon_mat(semantic_lexicon, target_words, NEW_ROWS, NEW_COLUMNS)

#returns the semantic lexicon as a matrix
#with each row representing a target word
#and each column representing a semantically related word from WordNet 
print(semantic_lexicon_matrix)

[[1 1 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 0 0
  1 1 0 0 0 1 1 0 1 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1
  1 0 1 0 1 0 0 1 1 1 1 1 1 0 0 1 0 1 0 1 0 0 1 0 1 1 0 1 0 1 1 1 1 1 1 1
  1 0 1 1 1 1 0 0 0 1 0 1 0 1 1 0 0 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 1 1
  0 0 1 0 1 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
  0 1 0 1 0 1 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
  0 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [47]:
#numpy.ndarray = dense matrix
print(type(semantic_lexicon_matrix))

<class 'numpy.ndarray'>
